<a href="https://colab.research.google.com/github/greyhound101/Brain_tumour/blob/master/0_loss_variational_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [ ]:
import copy
from datetime import timedelta, datetime
import imageio
import matplotlib.pyplot as plt
from matplotlib import cm
import multiprocessing
import numpy as np
import os
from pathlib import Path
import pytest
import scipy.ndimage as ndimage
from scipy.ndimage.interpolation import zoom
from skimage import measure, morphology, segmentation
from time import time, sleep
from tqdm import trange, tqdm
from torchvision import transforms
import warnings
import scipy

In [ ]:
import glob
image_paths=glob.glob('/content/gdrive/My Drive/ID*_segmented')
ids=[i.split('/')[-1].split('_')[0] for i in image_paths]

In [ ]:
ls=[]
for i in image_paths:
  ls.append(np.load(i+'/image.npy').reshape((40,256,256,1)))

In [ ]:
import pandas as pd
train = pd.read_csv('/content/gdrive/My Drive/lungs/train.csv') 
def get_tab(df):
    vector = [(df.Age.values[0] - 30) / 30] 
    
    if df.Sex.values[0] == 'male':
       vector.append(0)
    else:
       vector.append(1)
    
    if df.SmokingStatus.values[0] == 'Never smoked':
        vector.extend([0,0])
    elif df.SmokingStatus.values[0] == 'Ex-smoker':
        vector.extend([1,1])
    elif df.SmokingStatus.values[0] == 'Currently smokes':
        vector.extend([0,1])
    else:
        vector.extend([1,0])
    return np.array(vector) 
A = {} 
TAB = {} 
P = [] 
for i, p in tqdm(enumerate(ids)):
    sub = train.loc[train.Patient == p, :] 
    fvc = sub.FVC.values
    weeks = sub.Weeks.values
    c = np.vstack([weeks, np.ones(len(weeks))]).T
    a, b = np.linalg.lstsq(c, fvc)[0]
    
    A[p] = a
    TAB[p] = get_tab(sub)
    P.append(p)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
174it [00:00, 908.85it/s]


In [ ]:
from tensorflow.keras.utils import Sequence
class IGenerator(Sequence):
    def __init__(self, ids,image_paths, batch_size=32):
        self.keys = ids
        self.batch_size = batch_size
        self.image_paths=image_paths
        self.train_data = {}
        
    def __len__(self):
        return 2000
    
    def __getitem__(self, idx):
        self.x= []
        a, tab = [], [] 
        keys = np.random.choice(self.keys, size = self.batch_size)
        for k in keys:
            #try:
                chosen_idx=self.keys.index(k)
                chosen_path=self.image_paths[chosen_idx]
                img=chosen_path
                self.x.append(img)
        del img
        gc.collect()
        self.x = np.array(self.x)
        # self.x = np.expand_dims(self.x, axis=-1)
        return self.x , None
    def on_batch_end(self):
      del self.x
      gc.collect()

In [ ]:
train=IGenerator(ids,ls,1)
from tensorflow.keras.layers import *
from tensorflow.keras import *
from tensorflow.keras import backend as K
from tensorflow import keras
def build_model(shape=(40,256,256, 1), model_class=None,latent_dim=2):
  input_img = Input(shape=shape)
    
    
    
    
    
    
    #VAE
    

  x = layers.Conv3D(8, 3,
                  padding='same', 
                  activation='relu')(input_img)
  x = layers.MaxPooling3D()(x)
  x = BatchNormalization()(x)
  
  
  
  
  
  x = layers.Conv3D(8, 3,
                  padding='same', 
                  activation='relu',
                  )(x)
  x = layers.MaxPooling3D()(x)
  x = BatchNormalization()(x)



  x = layers.Conv3D(8, 3,
                  padding='same', 
                  activation='relu')(x)
  x = layers.MaxPooling3D()(x)
  x = BatchNormalization()(x)
  # # need to know the shape of the network here for the decoder
  # shape_before_flattening = K.int_shape(x)
  shape_before_flattening = K.int_shape(x)

  x = layers.Flatten()(x)
  x = layers.Dense(32, activation='relu')(x)


  # inp2 = Input(shape=(4,))
  # x1 = Concatenate()([inp2,x]) 
  # x1 = Dense(256,activation='relu')(x1)
  # x1 = BatchNormalization()(x1)
  # x1 = Dropout(0.3)(x1) 
  # x1 = Dense(256,activation='relu')(x1)
  # x1 = BatchNormalization()(x1)
  # x1 = Dropout(0.3)(x1) 
  # x1 = Dense(256,activation='relu')(x1)
  # x1 = BatchNormalization()(x1)
  # x1 = Dropout(0.3)(x1) 
  # x1 = Dense(1)(x1)
  # Two outputs, latent mean and (log)variance
  z_mu = layers.Dense(latent_dim)(x)
  z_log_sigma = layers.Dense(latent_dim)(x)

  # #sampling
  def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

  # sample vector from the latent distribution
  z = layers.Lambda(sampling)([z_mu, z_log_sigma])
  # construct a custom layer to calculate the loss
  # decoder takes the latent distribution sample as input






  #decoder
  decoder_input = layers.Input(K.int_shape(z)[1:])

  # Expand to 784 total pixels
  x = layers.Dense(np.prod(shape_before_flattening[1:]),
                 activation='relu')(decoder_input)

  # reshape
  x = layers.Reshape(shape_before_flattening[1:])(x)

  # use Conv2DTranspose to reverse the conv layers from the encoder
  x = layers.Conv3DTranspose(8, 3,
                           padding='same', 
                           activation='relu',
                           strides=(2, 2,2))(x)
  x = layers.Conv3D(8, 3,
                  padding='same', 
                  activation='sigmoid')(x)
  x = BatchNormalization()(x)



  x = layers.Conv3DTranspose(8, 3,
                           padding='same', 
                           activation='relu',
                           strides=(2, 2,2))(x)
  x = layers.Conv3D(8, 3,
                  padding='same', 
                  activation='sigmoid')(x)
  x = BatchNormalization()(x)
  
  
  
  
  x = layers.Conv3DTranspose(8, 3,
                           padding='same', 
                           activation='relu',
                           strides=(2, 2,2))(x)
  x = layers.Conv3D(8, 3,
                  padding='same', 
                  activation='sigmoid')(x)
  x = BatchNormalization()(x)
  # x = layers.Conv3DTranspose(8, 3,
  #                          padding='same', 
  #                          activation='relu',
  #                          strides=(2, 2,2))(x)
  # x = layers.Conv3D(8, 3,
  #                 padding='same', 
  #                 activation='sigmoid')(x)
  # x = BatchNormalization()(x)
  # x = layers.Conv3DTranspose(8, 3,
  #                          padding='same', 
  #                          activation='relu',
  #                          strides=(2, 2,2))(x)
  x = layers.Conv3D(1, 3,
                  padding='same', 
                  activation='sigmoid')(x)
  x = BatchNormalization()(x)
  x = tf.keras.layers.ZeroPadding3D(padding=4)(x)
  x = tf.keras.layers.Lambda(lambda x: x[:,4:44,4:260,4:260,:])(x)


  # decoder model statement
  decoder = Model(decoder_input, x)
  # apply the decoder to the sample from the latent distribution
  z_decoded = decoder(z)






  # construct a custom layer to calculate the loss
  class CustomVariationalLayer(keras.layers.Layer):

    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        # Reconstruction loss
        xent_loss = keras.metrics.mean_absolute_error(x, z_decoded)
        # KL divergence
        kl_loss = -5e-4 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
        return K.mean(xent_loss + kl_loss)

    # adds the custom loss to the class
    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

  # apply the custom loss to the input images and the decoded latent distribution sample
  y = CustomVariationalLayer()([input_img, z_decoded])


    
    
    
  
  model = Model(input_img ,y)
  return model


In [ ]:
import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [ ]:
mod=build_model()
mod.compile(optimizer='adam',loss=None)
mod.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40, 256, 256 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 40, 256, 256, 224         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling3d (MaxPooling3D)    (None, 20, 128, 128, 0           conv3d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 20, 128, 128, 32          max_pooling3d[0][0]              
_______________________________________________________________________________________

In [ ]:
import gc
mod.fit(train)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


   6/2000 [..............................] - ETA: 14:06 - loss: 0.0000e+00